In [25]:
import numpy as np
import pandas as pd

In [26]:
simRAO = 2000
nMTCD = 30000
MTCD_data = pd.read_csv(f'MTCD_data_{nMTCD}.csv', index_col=False)

In [27]:
data = np.zeros((simRAO, 4), dtype=int)
Preamble_status = pd.DataFrame(
    data,
    columns=['nRA', 'empty', 'collided', 'success'],
)
Preamble_status.index.name = 'system frame'

In [28]:
print(Preamble_status.shape)
Preamble_status.sample(5)

(2000, 4)


,nRA,empty,collided,success
system frame,,,,
1791,0,0,0,0
1188,0,0,0,0
872,0,0,0,0
1581,0,0,0,0
336,0,0,0,0


In [29]:
RA_data = MTCD_data[['RA_init', 'RA_first', 'RA_success', 'RA_transmit']]
RA_data.index.name = 'device_id'

In [30]:
print(RA_data.shape)
RA_data.sample(5)

(30000, 4)


,RA_init,RA_first,RA_success,RA_transmit
device_id,,,,
7505,416,416,-1,0
14971,300,300,-1,0
24892,506,506,-1,0
19989,213,213,-1,0
4439,385,385,-1,0


In [31]:
D2D_member = pd.DataFrame(
    columns=['group', 'Member', 'arrange', 'request', 'delay', 'transmit'],
)

# assign group
D2D_member['group'] = MTCD_data['clusters']

# assign type
D2D_member['Member'] = (D2D_member.index != D2D_member['group'])

# assign arrange
arrangePRB = 20
for group in D2D_member['group'].unique():
    members = D2D_member.loc[(D2D_member['group'] == group) &
                             (D2D_member['Member'])]
    n = 0
    for device_id, member in members.iterrows():
        D2D_member.at[device_id, 'arrange'] = n // arrangePRB
        n += 1

# initiate request
D2D_member['request'] = np.zeros(nMTCD, dtype=bool)

# initiate delay
D2D_member['delay'] = np.zeros(nMTCD, dtype=int)

# initiate transmit
D2D_member['transmit'] = np.zeros(nMTCD, dtype=int)

In [32]:
print(D2D_member.shape)
D2D_member.loc[D2D_member['Member'] == False].sample(5)

(30000, 6)


,group,Member,arrange,request,delay,transmit
device_id,,,,,,
316,316,False,NaN,False,0,0
127,127,False,NaN,False,0,0
145,145,False,NaN,False,0,0
84,84,False,NaN,False,0,0
93,93,False,NaN,False,0,0


In [33]:
D2D_group = pd.DataFrame(
    columns=['Header', 'class', 'seq_class', 'N_member', 'RA_request', 'arrange', 'HL']
)
D2D_group['Header'] = MTCD_data['clusters'].unique()

In [34]:
N_preamble = 54
N_group = len(D2D_group)
N_class = (N_group // N_preamble) + 1
seq = 0
ind = 0
print(N_group, N_class)

219 5


In [35]:
for group in range(N_group):
    D2D_group.at[group, 'class'] = group // N_preamble
    # assign inter-class sequence
    D2D_group.at[group, 'seq_class'] = seq
    seq = (seq+1) % N_preamble
D2D_group

,Header,class,seq_class,N_member,RA_request,arrange,HL
0,0,0,0,NaN,NaN,NaN,NaN
1,1,0,1,NaN,NaN,NaN,NaN
2,2,0,2,NaN,NaN,NaN,NaN
3,3,0,3,NaN,NaN,NaN,NaN
4,4,0,4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
214,1004,3,52,NaN,NaN,NaN,NaN
215,1056,3,53,NaN,NaN,NaN,NaN
216,1756,4,0,NaN,NaN,NaN,NaN
217,9805,4,1,NaN,NaN,NaN,NaN


In [36]:
D2D_group['N_member'] = D2D_member.groupby(['group']).size().values
D2D_group

,Header,class,seq_class,N_member,RA_request,arrange,HL
0,0,0,0,159,NaN,NaN,NaN
1,1,0,1,159,NaN,NaN,NaN
2,2,0,2,162,NaN,NaN,NaN
3,3,0,3,120,NaN,NaN,NaN
4,4,0,4,138,NaN,NaN,NaN
...,...,...,...,...,...,...,...
214,1004,3,52,96,NaN,NaN,NaN
215,1056,3,53,106,NaN,NaN,NaN
216,1756,4,0,119,NaN,NaN,NaN
217,9805,4,1,55,NaN,NaN,NaN


In [37]:
for i, header in enumerate(D2D_group['Header']):
    D2D_group.at[i, 'N_member'] = len(D2D_member)

In [38]:
BS_schedule = pd.DataFrame(
    columns=['class', 'TDCR'],
)

# assign class of every frame
BS_schedule['class'] = [f % N_class for f in range(simRAO)]

# initialize TDCR
BS_schedule['TDCR'] = np.zeros(simRAO, dtype=int)

In [39]:
BS_schedule

,class,TDCR
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
1995,0,0
1996,1,0
1997,2,0
1998,3,0


In [40]:
schedule = BS_schedule.loc[np.arange(0,16)]
schedule

,class,TDCR
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,0,0
6,1,0
7,2,0
8,3,0
9,4,0


In [41]:
s = schedule.loc[(schedule['class'] == 1)]
s

,class,TDCR
1,1,0
6,1,0
11,1,0


In [42]:
s.iat[0, 1] = 1
s

,class,TDCR
1,1,1
6,1,0
11,1,0


In [43]:
s.index[0]

1

In [44]:
BS_schedule.at[s.index[0], 'TDCR'] = 1235
BS_schedule

,class,TDCR
0,0,0
1,1,1235
2,2,0
3,3,0
4,4,0
...,...,...
1995,0,0
1996,1,0
1997,2,0
1998,3,0


In [48]:
devices = RA_data.loc[(RA_data['RA_init'] == 125) & (RA_data['RA_transmit'] <= 10)]
devices['RA_transmit'] += 1

In [49]:
devices

,RA_init,RA_first,RA_success,RA_transmit
device_id,,,,
139,125,125,-1,1
2183,125,125,-1,1
3230,125,125,-1,1
4765,125,125,-1,1
15464,125,125,-1,1
16923,125,125,-1,1
18483,125,125,-1,1
21849,125,125,-1,1
21861,125,125,-1,1


In [53]:
RA_data.iloc[139]

RA_init        125
RA_first       125
RA_success      -1
RA_transmit      0
Name: 139, dtype: int64

In [55]:
D2D_member['response'] = np.full(nMTCD, -1)

In [56]:
D2D_member

,group,Member,arrange,request,delay,transmit,response
device_id,,,,,,,
0,0,False,NaN,False,0,0,-1
1,1,False,NaN,False,0,0,-1
2,2,False,NaN,False,0,0,-1
3,3,False,NaN,False,0,0,-1
4,4,False,NaN,False,0,0,-1
...,...,...,...,...,...,...,...
29995,112,True,8,False,0,0,-1
29996,1305,True,6,False,0,0,-1
29997,138,True,8,False,0,0,-1


In [71]:
D2D_group['HL'] = np.random.choice(a=[False, True], size=len(D2D_group))
D2D_group

,Header,class,seq_class,N_member,RA_request,arrange,HL
0,0,0,0,30000,NaN,NaN,False
1,1,0,1,30000,NaN,NaN,False
2,2,0,2,30000,NaN,NaN,True
3,3,0,3,30000,NaN,NaN,False
4,4,0,4,30000,NaN,NaN,False
...,...,...,...,...,...,...,...
214,1004,3,52,30000,NaN,NaN,False
215,1056,3,53,30000,NaN,NaN,False
216,1756,4,0,30000,NaN,NaN,True
217,9805,4,1,30000,NaN,NaN,True


In [92]:
class_groups = D2D_group.loc[D2D_group['class'] == 3]
HL_groups = class_groups.loc[class_groups['HL'] == False]
print(len(class_groups), len(HL_groups))

54 25


In [95]:
class_groups

,Header,class,seq_class,N_member,RA_request,arrange,HL
162,295,3,0,30000,NaN,NaN,False
163,312,3,1,30000,NaN,NaN,False
164,313,3,2,30000,NaN,NaN,True
165,317,3,3,30000,NaN,NaN,True
166,14070,3,4,30000,NaN,NaN,False
167,323,3,5,30000,NaN,NaN,False
168,431,3,6,30000,NaN,NaN,True
169,553,3,7,30000,NaN,NaN,False
170,327,3,8,30000,NaN,NaN,False
171,832,3,9,30000,NaN,NaN,True


In [97]:
class_groups['Header']

162      295
163      312
164      313
165      317
166    14070
167      323
168      431
169      553
170      327
171      832
172     2455
173      383
174      349
175      600
176      353
177      356
178      996
179     4350
180      375
181      376
182      870
183      736
184      422
185     1602
186     1088
187     2476
188     5902
189      454
190     4819
191      462
192     5862
193      472
194     2863
195      487
196     2662
197     4585
198     1524
199    11624
200      540
201     4577
202     1476
203     7742
204     7811
205      579
206     1305
207     2798
208      637
209      741
210      783
211     8188
212    10106
213      963
214     1004
215     1056
Name: Header, dtype: int64

In [99]:
N_PRB_arrange = 20

In [102]:
D2D_member = pd.DataFrame(
    columns=['Header', 'Member', 'arrange', 'request', 'response', 'success', 'transmit'],
)

# assign group
D2D_member['Header'] = MTCD_data['clusters']

# assign type
D2D_member['Member'] = (D2D_member.index != D2D_member['Header'])

# assign arrange
for header in D2D_member['Header'].unique():
    members = D2D_member.loc[(D2D_member['Header'] == header) &
                             (D2D_member['Member'])]
    n = 0
    for device_id in members.index:
        D2D_member.at[device_id, 'arrange'] = n // N_PRB_arrange
        n += 1

# initialize request
D2D_member['request'] = np.zeros(nMTCD, dtype=bool)

# initialize response
D2D_member['response'] = np.zeros(nMTCD, dtype=bool)

# initialize success = -1
D2D_member['success'] = np.full(nMTCD, -1)

# initialize transmit
D2D_member['transmit'] = np.zeros(nMTCD, dtype=int)

In [109]:
# test dummy data
D2D_member['request'] = np.random.choice(a=[False, True], size=len(D2D_member))
D2D_member['response'] = np.random.choice(a=[False, True], size=len(D2D_member))
D2D_member['transmit'] = np.random.uniform(low=0,high=20,size=nMTCD).astype(int)
D2D_member.sample(10)

,Header,Member,arrange,request,response,success,transmit
device_id,,,,,,,
13252,3260,True,2,True,False,-1,0
17900,2455,True,3,False,True,-1,1
29612,143,True,9,True,True,-1,1
18450,162,True,4,False,True,-1,7
14779,431,True,3,True,True,-1,9
8584,162,True,2,True,False,-1,9
13277,64,True,1,False,True,-1,10
20715,46,True,5,False,False,-1,5
25454,268,True,5,False,True,-1,17


In [110]:
valid_group_members = D2D_member.loc[(D2D_member['Header'] == 0) & (D2D_member['response'])]
valid_group_members

,Header,Member,arrange,request,response,success,transmit
device_id,,,,,,,
0,0,False,NaN,False,True,-1,11
100,0,True,0,True,True,-1,10
186,0,True,0,True,True,-1,0
212,0,True,0,True,True,-1,9
307,0,True,0,False,True,-1,3
...,...,...,...,...,...,...,...
26797,0,True,6,True,True,-1,5
27115,0,True,7,False,True,-1,1
27863,0,True,7,False,True,-1,6


4109

In [123]:
device_id = valid_group_members['transmit'].idxmax()
valid_group_members = valid_group_members.drop(device_id)

In [124]:
device_id = valid_group_members['transmit'].idxmax()
valid_group_members.loc[device_id]


Header          0
Member       True
arrange         1
request     False
response     True
success        -1
transmit       19
Name: 6419, dtype: object

In [130]:
D2D_member.at[30001, 'transmit'] = 5

In [131]:
D2D_member

,Header,Member,arrange,request,response,success,transmit
device_id,,,,,,,
0,0.0,False,NaN,False,True,-1.0,11.0
1,1.0,False,NaN,True,True,-1.0,14.0
2,2.0,False,NaN,True,False,-1.0,18.0
3,3.0,False,NaN,False,False,-1.0,15.0
4,4.0,False,NaN,True,True,-1.0,16.0
...,...,...,...,...,...,...,...
29996,1305.0,True,6,True,True,-1.0,8.0
29997,138.0,True,8,False,False,-1.0,15.0
29998,227.0,True,3,False,False,-1.0,8.0


In [133]:
2 in [0,1,1,2,1]

True